In [1]:
datapath_root="../Desktop/ToParse"
!ls ../Desktop/ToParse

2022-03-16 2022-03-20 Icon?


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import sys
from math import ceil, floor
import os
import copy
import csv
import numpy as np
from metpy.io import Level2File
from metpy.plots import add_timestamp, ctables

In [3]:
def getResized(array,x,y):
    #min = 1
    #maxim = 0
    mx=int(floor(len(array)/x))
    my=int(floor(len(array[0])/y*5/8)) #exclude outer 3/8 of data *5/8
    output=[]
    for i in range(x):
        line=[]
        for j in range(y):
            sum=0
            for k in range(i*mx,(i+1)*mx):
                for l in range(j*my,(j+1)*my):
                    sum+=np.nan_to_num(array[k][l], nan=0)
            #print("sum", sum)
            line.append(sum/mx/my)
            #if min > sum/mx/my:
             #   min = sum/mx/my
            #elif maxim < sum/mx/my:
             #   maxim = sum/mx/my
        output.append(line)
    return output

In [4]:
station=""
stations = ["KABR", "KIND", "KJKL", "KDHL"]
years=["2022"]
months = ["03"]
#days = ["01", "02", "03", "04", "05", "06", "07", "08", "09","10","11", "12", "13", "14", "15", "16", "17", "18", "19","20","21", "22", "23", "24", "25", "26", "27", "28", "29","30","31"]
days = ["16", "20"]

In [5]:
def extract_nexrad_level2_data(nexrad_level2_file, sweep=0):
    f = Level2File(nexrad_level2_file)

    az = np.array([ray[0].az_angle for ray in f.sweeps[sweep]])

    ref_hdr = f.sweeps[sweep][0][4][b'REF'][0]
    ref_range = np.arange(ref_hdr.num_gates) * ref_hdr.gate_width + ref_hdr.first_gate
    ref = np.array([ray[4][b'REF'][1] for ray in f.sweeps[sweep]])

    rho_hdr = f.sweeps[sweep][0][4][b'RHO'][0]
    rho_range = (np.arange(rho_hdr.num_gates + 1) - 0.5) * rho_hdr.gate_width + rho_hdr.first_gate
    rho = np.array([ray[4][b'RHO'][1] for ray in f.sweeps[sweep]])

    phi_hdr = f.sweeps[sweep][0][4][b'PHI'][0]
    phi_range = (np.arange(phi_hdr.num_gates + 1) - 0.5) * phi_hdr.gate_width + phi_hdr.first_gate
    phi = np.array([ray[4][b'PHI'][1] for ray in f.sweeps[sweep]])

    zdr_hdr = f.sweeps[sweep][0][4][b'ZDR'][0]
    zdr_range = (np.arange(zdr_hdr.num_gates + 1) - 0.5) * zdr_hdr.gate_width + zdr_hdr.first_gate
    zdr = np.array([ray[4][b'ZDR'][1] for ray in f.sweeps[sweep]])
    nexrad_level2_data = {"timestamp": f.dt}

    for var_data, var_range, lbl in zip((ref, rho, zdr, phi),
                                        (ref_range, rho_range, zdr_range, phi_range),
                                        ('REF (dBZ)', 'RHO', 'ZDR (dBZ)', 'PHI')):
        # Turn into an array, then mask
        data = np.ma.array(var_data)
        data[np.isnan(data)] = np.ma.masked

        # Convert az,range to x,y
        xlocs = var_range * np.sin(np.deg2rad(az[:, np.newaxis]))
        ylocs = var_range * np.cos(np.deg2rad(az[:, np.newaxis]))
        nexrad_level2_data[lbl] = {"x_data": xlocs, "y_data": ylocs, "z_data": var_data}

    return nexrad_level2_data


In [6]:
from datetime import datetime

ref_min = 1
ref_max = 1
cor_min = 1
cor_max = 1
zdr_min = 1
zdr_max = 1
phi_min = 1
phi_max = 1

for year in years:
    for month in months:
        for day in days:
            day_path="/"+year+"-"+month+"-"+day
            folder_datapath=datapath_root+day_path
            directory_exists=True
            output_filepath=datapath_root+day_path+"txt"
                    #Check to make sure output_filepath is consistent with what youve been using
            
            file_exists=os.path.exists(output_filepath)
            if(file_exists):
                print("file already exists: ",output_filepath)
            try:
                filenames=os.listdir(folder_datapath)
            except:
                print("directory does not exist: "+folder_datapath)
                directory_exists=False
            if(directory_exists and not file_exists):
                for filename in filenames:
                    if(len(filename)==23):
                        print(filename)
                        datapath=folder_datapath+"/"+filename
                        datapoint=extract_nexrad_level2_data(datapath)
                        ref = np.array(getResized(datapoint['REF (dBZ)']['z_data'],180,180))
                        if (ref.min() < ref_min):
                            ref_min = ref.min()
                        if (ref.max() > ref_max):
                            ref_max = ref.max()
                        zdr = np.array(getResized(datapoint['ZDR (dBZ)']['z_data'],180,180))
                        if (zdr.min() < zdr_min):
                            zdr_min = zdr.min()
                        if (zdr.max() > zdr_max):
                            zdr_max = zdr.max()
                        phi = np.array(getResized(datapoint['PHI']['z_data'],180,180))
                        if (phi.min() < phi_min):
                            phi_min = phi.min()
                        if (phi.max() > phi_max):
                            phi_max = phi.max()
                        cor = np.array(getResized(datapoint['RHO']['z_data'],180,180))
                        if (cor.min() < cor_min):
                            cor_min = cor.min()
                        if (cor.max() > cor_max):
                            cor_max = cor.max()
                    
                        print("ref: ", ref_min, ref_max)
                        print("cor: ", cor_min, cor_max)
                        print("zdr: ", zdr_min, zdr_max)
                        print("phi: ", phi_min, phi_max)
            
                        file = open(output_filepath, 'a+', newline ='')
                        with file:
                            rlabel="R"
                            zlabel="Z"
                            clabel="C"
                            plabel="P"
                            write = csv.writer(file)
                            write.writerow(rlabel)
                            write.writerows(ref)
                            write.writerows(clabel)
                            write.writerows(cor)
                            write.writerows(zlabel)
                            write.writerows(zdr)
                            write.writerows(plabel)
                            write.writerows(phi)                        
                        print("Finished: ", datetime.now())
print("Set done.")

KIND20220316_222656_V06
ref:  -11.666666666666666 27.979166666666668
cor:  0.0 1.0247916666666665
zdr:  -11.748046875 9.802734375
phi:  0.0 250.49803412493463
Finished:  2022-12-07 00:04:31.180765
KIND20220317_005607_V06
ref:  -19.5625 27.979166666666668
cor:  0.0 1.0247916666666665
zdr:  -11.748046875 9.802734375
phi:  0.0 266.4530421927144
Finished:  2022-12-07 00:05:12.073421
KIND20220316_220533_V06
ref:  -19.5625 29.645833333333332
cor:  0.0 1.0312499999999998
zdr:  -11.748046875 10.236328125
phi:  0.0 266.4530421927144
Finished:  2022-12-07 00:05:52.288892
KIND20220316_230245_V06
ref:  -19.5625 34.25
cor:  0.0 1.0312499999999998
zdr:  -11.748046875 10.236328125
phi:  0.0 286.242541149712
Finished:  2022-12-07 00:06:32.120860
KIND20220317_002753_V06
ref:  -19.5625 34.25
cor:  0.0 1.0312499999999998
zdr:  -12.140625 10.236328125
phi:  0.0 286.242541149712
Finished:  2022-12-07 00:07:12.332466
KIND20220316_221944_V06
ref:  -19.5625 34.25
cor:  0.0 1.0354166666666664
zdr:  -12.140625 

ref:  -22.395833333333332 34.25
cor:  0.0 1.0354166666666664
zdr:  -12.46484375 11.392578125
phi:  0.0 329.39158230539266
Finished:  2022-12-07 00:35:38.788368
KIND20220321_001401_V06
ref:  -22.395833333333332 34.25
cor:  0.0 1.0354166666666664
zdr:  -12.46484375 11.392578125
phi:  0.0 329.39158230539266
Finished:  2022-12-07 00:36:18.258565
KIND20220321_002054_V06
ref:  -22.395833333333332 34.25
cor:  0.0 1.0354166666666664
zdr:  -12.46484375 11.392578125
phi:  0.0 329.39158230539266
Finished:  2022-12-07 00:36:58.367173
KIND20220321_004142_V06
ref:  -22.395833333333332 34.25
cor:  0.0 1.0354166666666664
zdr:  -12.46484375 11.392578125
phi:  0.0 329.39158230539266
Finished:  2022-12-07 00:37:39.193675
Set done.
